# Задача 2. Определение стоимости авто
Ключевая метрика RMSE -  среднеквадратичная ошибка. Чтобы сдать проект значение метрики должно быть не более 2500.

# Импортируем необходимые библиотеки 

In [1]:
import pandas as pd

# Подготовка данных

In [2]:
name_columns = [
    'DateCrawled',
    ' Price',
    'VehicleType',
    'RegistrationYear',
    'Gearbox',
    'Power',
    'Model',
    'Kilometer',
    'RegistrationMonth',
    'FuelType',
    'Brand',
    'Repaired',
    'DateCreated',
    'NumberOfPictures',
    'PostalCode',
    'LastSeen'
]

In [4]:
df = pd.read_csv("autos.csv",names=name_columns,header=0)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1    Price             354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [8]:
df.head

<bound method NDFrame.head of                 DateCrawled   Price  VehicleType  RegistrationYear Gearbox  \
0       2016-03-24 11:52:17     480          NaN              1993  manual   
1       2016-03-24 10:58:45   18300        coupe              2011  manual   
2       2016-03-14 12:52:21    9800          suv              2004    auto   
3       2016-03-17 16:54:04    1500        small              2001  manual   
4       2016-03-31 17:25:20    3600        small              2008  manual   
...                     ...     ...          ...               ...     ...   
354364  2016-03-21 09:50:58       0          NaN              2005  manual   
354365  2016-03-14 17:48:27    2200          NaN              2005     NaN   
354366  2016-03-05 19:56:21    1199  convertible              2000    auto   
354367  2016-03-19 18:57:12    9200          bus              1996  manual   
354368  2016-03-20 19:41:08    3400        wagon              2002  manual   

        Power        Model  Kilom